In [ ]:
# creds: 
# data collection
# https://www.youtube.com/watch?v=gI6wXYppiNQ

# data cleaning
# https://www.youtube.com/watch?v=TF7fAw95U0o

In [ ]:
!pip install google-play-scraper
!pip install nltk
from google_play_scraper import app, Sort, reviews_all, reviews

In [ ]:
import pandas as pd
import numpy as np
import string

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mazel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
result, continuation_token = reviews(
    'com.grabtaxi.passenger',
    lang='en_SG', # defaults to 'en'
    country='sg', # defaults to 'us'
    sort=Sort.NEWEST, # defaults to Sort.NEWEST
#     sort=Sort.MOST_RELEVANT, # Use if you need to sort by highest thumbsUpCount
    count=10000, # defaults to 100
#     filter_score_with=5 # defaults to None(means all score)
)

In [ ]:
scrapeddata = pd.DataFrame(np.array(result),columns=['review'])
scrapeddata = scrapeddata.join(pd.DataFrame(scrapeddata.pop('review').tolist()))

In [ ]:
scrapeddata.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,324ce550-8437-49bc-a3ca-45446064a2ec,Mala Luhan,https://play-lh.googleusercontent.com/a-/AD5-W...,"After update, very poor performance, taking 30...",1,0,5.238.0,2023-02-11 08:54:07,"Hi, we are always looking to improve the exper...",2023-02-11 15:08:28
1,03b624f5-d700-41bc-8446-13093dcb1472,Saiful Azmie,https://play-lh.googleusercontent.com/a-/AD5-W...,Bad app ..,1,0,5.239.0,2023-02-11 08:53:51,We are sad to know that your experience on the...,2023-02-11 15:08:52
2,18cd984c-86a9-4f99-aabc-f2342a236358,Jayeson Lee-Steere,https://play-lh.googleusercontent.com/a/AEdFTp...,"Very friendly and helpful - very nice, well ma...",5,0,5.239.0,2023-02-11 08:40:39,None,NaT
3,f275a70e-c0e5-442d-b0bb-26a7645684b0,livvy Bardoe,https://play-lh.googleusercontent.com/a/AEdFTp...,I had to remove the grab app as I was only loo...,2,0,5.239.0,2023-02-11 08:23:40,None,NaT
4,f8e8b2bf-d1d6-4297-b9a4-b38b3e801b87,Andrea Pagani,https://play-lh.googleusercontent.com/a/AEdFTp...,It's embarassing that in Singapore the waiting...,2,0,5.239.0,2023-02-11 08:16:02,None,NaT


In [ ]:
filteredData = scrapeddata[['content','score','at','replyContent','repliedAt']]

filteredData.head() 

,content,score,at,replyContent,repliedAt
0,"After update, very poor performance, taking 30...",1,2023-02-11 08:54:07,"Hi, we are always looking to improve the exper...",2023-02-11 15:08:28
1,Bad app ..,1,2023-02-11 08:53:51,We are sad to know that your experience on the...,2023-02-11 15:08:52
2,"Very friendly and helpful - very nice, well ma...",5,2023-02-11 08:40:39,None,NaT
3,I had to remove the grab app as I was only loo...,2,2023-02-11 08:23:40,None,NaT
4,It's embarassing that in Singapore the waiting...,2,2023-02-11 08:16:02,None,NaT


In [ ]:
def removepunctuation(text):
    p = set(string.punctuation) # p take all the punctuations, punctuations displayed below
    text=text.lower() # convert text to lower case
    words=text.split() # split the text into words
    ctext=[]
    for i in range(10): #adding numbers from 0 - 9 to p 
        p.add(str(i))
    for i in words:
        t=''.join([x for x in i.encode("ascii","ignore").decode("ascii") if x not in p]) # ignoring non ascii charecters and numbers
        ctext.append(t)
    return " ".join([i for i in ctext]) # joining the cleaned words to text

def normalise_case(uncleaned):
    cleaned=[]
    for i in uncleaned:
        try:
            ctxt=removepunctuation(i)
            if len(ctxt)==0: raise()
            cleaned.append(ctxt)
        except:
            cleaned.append(np.nan)
    
    cleaned = pd.Series(cleaned).astype(str).apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))
    cleaned = cleaned.replace(r'^\s*$', np.nan, regex=True)
    return cleaned
        
stop_words = stopwords.words('english')

filteredData['cleanedcontent'] = normalise_case(filteredData.content)  # attaching the cleaned data as a new column to the dataset
filteredData['cleanedreplyContent'] = normalise_case(filteredData.replyContent)  # attaching the cleaned data as a new column to the dataset

<ipython-input-20-9dd371f7e965>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filteredData['cleanedcontent'] = normalise_case(filteredData.content)  # attaching the cleaned data as a new column to the dataset
<ipython-input-20-9dd371f7e965>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filteredData['cleanedreplyContent'] = normalise_case(filteredData.replyContent)  # attaching the cleaned data as a new column to the dataset


In [ ]:
filteredData.head()
# filteredData.info()
# filteredData['at'].dt.day

,content,score,at,replyContent,repliedAt,cleanedcontent,cleanedreplyContent
0,"After update, very poor performance, taking 30...",1,2023-02-11 08:54:07,"Hi, we are always looking to improve the exper...",2023-02-11 15:08:28,update poor performance taking minutes search ...,hi always looking improve experience users rea...
1,Bad app ..,1,2023-02-11 08:53:51,We are sad to know that your experience on the...,2023-02-11 15:08:52,bad app,sad know experience app satisfactory please sh...
2,"Very friendly and helpful - very nice, well ma...",5,2023-02-11 08:40:39,None,NaT,friendly helpful nice well maintained vehicle ...,nan
3,I had to remove the grab app as I was only loo...,2,2023-02-11 08:23:40,None,NaT,remove grab app looking food knew car number w...,nan
4,It's embarassing that in Singapore the waiting...,2,2023-02-11 08:16:02,None,NaT,embarassing singapore waiting time reduced min...,nan


In [ ]:
filteredData.to_csv("playstorescrapping.csv", index = False)  #Save the file as CSV